# Anomaly Detection Through Clustering Exercises

In [1]:
# tabular manipulation:
import numpy as np
import pandas as pd
# visualization:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt
import seaborn as sns
# sklearn for scaling and clustering:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
# environment:
from env import host, user, password

<hr style="border:2px solid black"> </hr>

### Clustering - DBSCAN

1. Use DBSCAN to detect anomalies in other products from the customers dataset. The database name is grocery_db.
2. Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.
3. Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

In [2]:
def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

<hr style="border:2px solid black"> </hr>

#### #1. Use DBSCAN to detect anomalies in other products from the customers dataset. 
The database name is grocery_db.

In [3]:
url = get_db_url("grocery_db")

sql = """
select *
from grocery_customers
"""

df = pd.read_sql(sql, url, index_col="customer_id")
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [5]:
#select features we wish to examine
selected_feats = ['Delicassen','Frozen', 'Detergents_Paper']

In [6]:
#create a scaler
#make it
minmax = MinMaxScaler()

#fit it/ #use it
scaled_features = minmax.fit_transform(df[selected_feats])

In [7]:
scaled_features

array([[0.02784731, 0.0031063 , 0.0654272 ],
       [0.03698373, 0.02854842, 0.08058985],
       [0.16355861, 0.03911643, 0.08605232],
       ...,
       [0.03888194, 0.00677142, 0.36346267],
       [0.04426366, 0.01664914, 0.00404174],
       [0.00102211, 0.00065742, 0.01161082]])

In [8]:
#create variable for scaled data
scaled_cols = [col +'_scaled' for col in selected_feats]

In [9]:
#create the dataframe in a scaled version
scaled_df = pd.DataFrame(scaled_features, index=df.index, columns=scaled_cols)

In [10]:
#concat info back into the original df
df = df.merge(scaled_df, on= df.index)

In [11]:
#take a look at the merged df, will have scaled data and normal
df.head()

,key_0,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Delicassen_scaled,Frozen_scaled,Detergents_Paper_scaled
0,0,2,3,12669,9656,7561,214,2674,1338,0.027847,0.003106,0.065427
1,1,2,3,7057,9810,9568,1762,3293,1776,0.036984,0.028548,0.080590
2,2,2,3,6353,8808,7684,2405,3516,7844,0.163559,0.039116,0.086052
3,3,1,3,13265,1196,4221,6404,507,1788,0.037234,0.104842,0.012346
4,4,2,3,22615,5410,7198,3915,1777,5185,0.108093,0.063934,0.043455


In [12]:
#remove column key_0
df = df.drop(columns=['key_0'])

In [14]:
#make sure its dropped
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Delicassen_scaled,Frozen_scaled,Detergents_Paper_scaled
0,2,3,12669,9656,7561,214,2674,1338,0.027847,0.003106,0.065427
1,2,3,7057,9810,9568,1762,3293,1776,0.036984,0.028548,0.080590
2,2,3,6353,8808,7684,2405,3516,7844,0.163559,0.039116,0.086052
3,1,3,13265,1196,4221,6404,507,1788,0.037234,0.104842,0.012346
4,2,3,22615,5410,7198,3915,1777,5185,0.108093,0.063934,0.043455


### create DBSCAN model

In [29]:
#make it
dbsc = DBSCAN(eps= 0.2, min_samples=30)

#fit it
dbsc.fit(scaled_features)

DBSCAN(eps=0.2, min_samples=30)

In [30]:
#use it
dbsc.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [31]:
#assign a column to the labels
df['labels'] = dbsc.labels_

In [32]:
#this will show the selected features and the clusters they belong to
df[selected_feats + ['labels']].head()

,Delicassen,Frozen,Detergents_Paper,labels
0,1338,214,2674,0
1,1776,1762,3293,0
2,7844,2405,3516,0
3,1788,6404,507,0
4,5185,3915,1777,0


In [33]:
#take a look at the cluster counts
df.labels.value_counts()

#this shows that there is one cluster
#out of 440 entries, 8 of them are outliers

 0    432
-1      8
Name: labels, dtype: int64

### Explore

In [34]:
#Examine this on a 3D scale
#matplotlib figure
fig = plt.figure(1, figsize=(10,10))
#wrap that figure in 3D figure
ax = Axes3D(fig)
                 
#ax.scatter(x,y,z- c=color/hue)
ax.scatter(df.Delicassen, 
           df.Frozen, 
           df.Detergents_Paper,
           c=df.labels,
          edgecolor='k')
                 
#remove anything within axis labels
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

#label your axis                 
ax.set_xlabel('Delicassen')
ax.set_ylabel('Frozen')
ax.set_zlabel('Detergents_Paper')


Text(0.5, 0, 'Detergents_Paper')

### Takeaways:
- there appear to be 8 outliers
- only one cluster was created

<hr style="border:2px solid black"> </hr>

#### #2. Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.

In [51]:
url = get_db_url("curriculum_logs")

sql = """
select *
from logs
join cohorts on logs.cohort_id= cohorts.id
"""

df = pd.read_sql(sql, url)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2


In [52]:
#remove column 'id'- it is redundent, same as cohort_id
df = df.drop(columns=['id'])

In [53]:
#look at size of df
df.shape

(847330, 14)

In [54]:
#x = date
#y = count

In [55]:
df.groupby(['date', 'user_id']).count()

time  path  cohort_id  ip  name  slack  start_date  \
date       user_id                                                       
2018-01-26 1           7     7          7   7     7      7           7   
           2          17    17         17  17    17     17          17   
           3          22    22         22  22    22     22          22   
           4          14    14         14  14    14     14          14   
           5          14    14         14  14    14     14          14   
...                  ...   ...        ...  ..   ...    ...         ...   
2021-04-21 976         3     3          3   3     3      3           3   
           977         4     4          4   4     4      4           4   
           978         6     6          6   6     6      6           6   
           979         5     5          5   5     5      5           5   
           981         8     8          8   8     8      8           8   

                    end_date  created_at  updated_at  deleted_at  program_id  
date       user_id                                                            
2018-01-26 1               7           7           7           0           7  
           2              17          17          17           0          17  
           3              22          22          22           0          22  
           4              14          14          14           0          14  
           5              14          14          14           0          14  
...                      ...         ...         ...         ...         ...  
2021-04-21 976             3           3           3           0           3  
           977             4           4           4           0           4  
           978             6           6           6           0           6  
           979             5           5           5           0           5  
           981             8           8           8           0           8  

[72195 rows x 12 columns]

In [ ]:
#create categorical features


In [56]:
id_counts = df.groupby(['user_id'])['date','cohort_id','path'].nunique()
cohort_counts = df.groupby(['cohort_id'])['path','user_id','date'].nunique()
first_access = df.groupby(['user_id'])['date'].min()
id_by_first_access_date = pd.DataFrame({'first_access_date': first_access}).reset_index().groupby('first_access_date').count()

<ipython-input-56-6d64d3f2285b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  id_counts = df.groupby(['user_id'])['date','cohort_id','path'].nunique()
<ipython-input-56-6d64d3f2285b>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cohort_counts = df.groupby(['cohort_id'])['path','user_id','date'].nunique()


In [57]:
id_by_first_access_date

,user_id
first_access_date,
2018-01-26,49
2018-01-27,2
2018-01-28,1
2018-01-29,7
2018-01-30,6
...,...
2021-04-08,1
2021-04-11,1
2021-04-12,26


In [59]:
id_counts[id_counts.cohort_id > 1]

,date,cohort_id,path
user_id,,,
1,637,2,976
11,829,2,871
25,114,2,154
37,43,2,156
40,102,2,120
41,221,2,186
53,772,2,333
64,744,3,436
88,35,3,113


<hr style="border:2px solid black"> </hr>

#### #3. Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).